In [1]:
import pandas as pd
import os

from pymongo import MongoClient

### Limpa Dataset de tipos de infrações

In [62]:
df_tipos_infracoes_2018 = pd.read_excel("/home/iago/Documentos/TCC1/tcc_scripts/datasets/tipos-infracoes-2018.xlsx")
df_tipos_infracoes_2018 = df_tipos_infracoes_2018.rename(columns = {'Unnamed: 0': 'TIPO DE INFRACAO'})

df_tipos_infracoes_2018.head(n = 20)

,TIPO DE INFRACAO,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,ALCOOLEMIA,912,764,324,356,742,334,759,807,892,567,571,610
1,INABILITADO,623,367,169,271,443,179,473,540,448,404,286,289
2,VEÍCULO NÃO LICENCIADO,1602,808,439,349,927,397,922,917,758,877,700,747
3,DIRIGIR UTILIZANDO TELEFONE CELULAR,709,484,202,366,405,188,420,456,377,412,273,321
4,CINTO DE SEGURANÇA,4372,3431,941,1223,2521,1222,3098,3143,2028,3034,2686,2587
5,CADEIRINHA,138,169,42,53,115,52,116,137,96,132,106,97
6,FAIXA DE PEDESTRE,635,482,239,303,568,268,482,452,446,526,503,399
7,ESTACIONAMENTO,21618,12398,5960,7318,8984,4159,11131,10767,10027,11737,9622,8606
8,AVANÇO DE SEMÁFORO,8475,6488,4718,7690,8721,2995,6467,7214,5728,6497,7878,3780


### Limpa Dataset de acidentes com vítimas fatais

In [63]:
df_acidentes_fatais_2008_2017 = pd.read_excel("/home/iago/Documentos/TCC1/tcc_scripts/datasets/acidentes/acidentes-fatais-2008-2017.xlsx")
df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop("Fatal", axis = 1)
df_acidentes_fatais_2008_2017['Ano'] = 0


for ano in range(2008, 2018):
    df_acidentes_fatais_2008_2017.loc[df_acidentes_fatais_2008_2017['Ano ' + str(ano)] == 1.0, 'Ano'] = str(ano)
    df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop('Ano ' + str(ano), axis = 1)

df_acidentes_fatais_2008_2017.head()

,Região Administrativa,Nome da Via e Trecho,Descrição do Endereço,Natureza,Data,Dia da Semana,Hora,Faixa Horária (6x6),Nº de Vit Fatais,Nº de Vit Feridas,Veículos Múltiplos,Ano
0,ÁGUAS CLARAS,ALAMEDA DOS EUCALIPTOS,"ALAMEDA DOS EUCALIPTOS, QD 107, PRÓX. AO RESTA...",CHOQ OBJ FIXO,05/03/2015,5-QUI,20:00,18:00 - 23:59,1.0,-,MOTOCICLETA,2015
1,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, PRÓX. À UNIVERSIDADE CATÓLIC...",CAPOTAMENTO,11/05/2015,2-SEG,06:00,06:00 - 11:59,1.0,1,CAMINHONETA,2015
2,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO LOTE 48",COLISÃO,08/04/2011,6-SEX,18:10,18:00 - 23:59,1.0,-,CAMINHÃO x BICICLETA,2011
3,ÁGUAS CLARAS,AV. ÁGUAS CLARAS,"AV. ÁGUAS CLARAS, QS 08, EM FRENTE AO POSTO PO...",COLISÃO TRASEIRA,24/12/2009,5-QUI,11:40,06:00 - 11:59,1.0,-,ÔNIBUS x BICICLETA,2009
4,ÁGUAS CLARAS,AV. ARNIQUEIRA,"AV. ARNIQUEIRA, QD. 04, PRÓX. Á CHÁC. 81",ATROP PEDESTRE,17/11/2011,5-QUI,17:00,12:00 - 17:59,1.0,-,CAMINHONETA,2011


### Limpa Dataset de pedestres mortos em trechos não semaforizados

In [8]:
df_pedestres_mortos_trechos_ns_2008_2017 = pd.read_excel("/home/iago/Documentos/TCC1/tcc_scripts/datasets/pedestres-mortos-trechos-ns-2008-2017.xlsx")

for ano in range(2008, 2018):
    df_pedestres_mortos_trechos_ns_2008_2017 = df_pedestres_mortos_trechos_ns_2008_2017.drop(ano, axis = 1)

df_pedestres_mortos_trechos_ns_2008_2017.head()

,JURISDIÇÃO,REGIÃO ADMINISTRATIVA,ENDEREÇO,NATUREZA,DATA,DIA DA SEMANA,HORA,VEÍCULO,SEXO,IDADE
0,VIA URBANA,BRASÍLIA,"L1 SUL, SQS 411, FAIXA DE PED",ATROP PEDESTRE,26/11/2017,1-DOM,19:30,AUTOMÓVEL,MASCULINO,82
1,VIA URBANA,BRASÍLIA,"W1 SUL, QUADRA 114 PROXIMO A CLS 314, AO LADO ...",ATROP PEDESTRE,29/04/2008,3-TER,10:00,CAMINHONETE,FEMININO,58
2,VIA URBANA,CEILÂNDIA,"ELMO SEREJO, ADE QD 01, PRÓX. À RAGNA ROCK MUS...",ATROP PEDESTRE,15/11/2017,4-QUA,04:50,AUTOMÓVEL,FEMININO,31
3,VIA URBANA,CEILÂNDIA,"VIA M2 - NORTE, EQNM 22/24, PRÓX. AO QUIOSQUE ...",ATROP PEDESTRE,16/08/2014,7-SAB,22:45,MOTO,FEMININO,64
4,VIA URBANA,CEILÂNDIA,"VIA M2 - NORTE, QNM 12, PRÓX. AO OBCURSOS",ATROP PEDESTRE,14/01/2015,4-QUA,09:00,AUTOMÓVEL,MASCULINO,84


### Limpa Dataset de Cursos oferecidos pelo Detran

In [18]:
df_cursos_oferecidos_detran_2018 = pd.read_excel("/home/iago/Documentos/TCC1/tcc_scripts/datasets/cursos_oferecidos_detran.xlsx")
df_cursos_oferecidos_detran_2018 = df_cursos_oferecidos_detran_2018.fillna(0)

for mes in df_cursos_oferecidos_detran_2018.columns[3:]:
    df_cursos_oferecidos_detran_2018[mes] = df_cursos_oferecidos_detran_2018[mes].astype(int)

df_cursos_oferecidos_detran_2018.head()

,CURSO,RESULTADO,TOTAL,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,RECICLAGEM PARA CONDUTOR INFRATOR,APROVADO,2179,160,167,180,39,242,140,218,228,223,186,237,159
1,RECICLAGEM PARA CONDUTOR INFRATOR,REPROVADO,556,39,50,43,10,63,24,60,43,53,67,60,44
2,RETESTE RECICLAGEM PARA CONDUTOR INFRATOR,APROVADO,489,19,33,72,23,44,40,44,51,32,57,36,38
3,RETESTE RECICLAGEM PARA CONDUTOR INFRATOR,REPROVADO,137,11,10,20,11,8,6,8,14,21,13,7,8
4,MECÂNICA PARA MULHERES,SEM AVALIACAO,50,0,0,0,16,10,0,0,24,0,0,0,0


### Limpa Dataset de campanhas oferecidas pelo Detran

In [38]:
df_campanhas_educativas_2012_2019 = pd.read_excel("/home/iago/Documentos/TCC1/tcc_scripts/datasets/campanhas_educativas_2012_2019.xlsx")

for column in df_campanhas_educativas_2012_2019.columns[6:10]:
    df_campanhas_educativas_2012_2019[column] = df_campanhas_educativas_2012_2019[column].fillna(0)
    df_campanhas_educativas_2012_2019[column] = df_campanhas_educativas_2012_2019[column].astype(int)

df_campanhas_educativas_2012_2019['Região Administrativa do Evento da Campanha'] = df_campanhas_educativas_2012_2019['Região Administrativa do Evento da Campanha'].fillna("--")

df_campanhas_educativas_2012_2019.head()

,Campanha,Objetivos da Campanha,Data de Início da campanha,Data Final da Campanha,Local do Evento da Campanha,Região Administrativa do Evento da Campanha,Qtd de Motoristas Abordados no Evento,Qtd de Pedestres Abordados no Evento,Qtd De Motociclistas Abordados no Evento,Qtd de Ciclistas Abordados no Evento,Data Inicial do Evento da Campanha,Hora Inicial do Evento da Campanha,Data Final do Evento da Campanha,Hora Final do Evento da Campanha,Relatório do Evento,Rodovia
0,GECED - BLITZEN NOTURNAS,Campanha Educativa de entrega de folder Alcool...,2015-11-27 00:00:00.000,2015-10-27 00:00:00.000,BR 070,--,700,0,0,0,2012-01-06,00:00:00:00,2012-01-06,00:00:00:00,NaN,NaN
1,Operação Funil,"Álcool e direção, conscientização de motoristas.",2012-01-10 00:00:00.000,2012-01-10 00:00:00.000,BR 040 - Posto da Polícia Rodoviária Federal,--,300,0,0,0,2012-01-10,00:00:00:00,2012-01-10,00:00:00:00,NaN,NaN
2,Operação Funil,Abordagem de carros. Operação iniciada ás 15:3...,2012-01-12 00:00:00.000,2012-01-12 00:00:00.000,BR 251/ DF 001 - Saída para Unaí,--,448,0,0,0,2012-01-12,00:00:00:00,2012-01-12,00:00:00:00,NaN,NaN
3,Operação Funil,Abordagem de carros iniciando ás 15h e finaliz...,2012-01-19 00:00:00.000,2012-12-30 00:00:00.000,CPRV BR 060 - Samambaia,--,550,0,0,0,2012-01-19,00:00:00:00,2012-01-19,00:00:00:00,NaN,NaN
4,Operação Funil,Abordagem de carros.,2012-01-06 00:00:00.000,2012-12-30 00:00:00.000,BR 020 Posto da BPRv,--,0,0,0,0,2012-01-20,00:00:00:00,2012-01-20,00:00:00:00,NaN,NaN


### Funções para acesso ao banco

In [42]:
client = MongoClient("mongodb://localhost:27017/")
db = client['dados_de_transito']


def dataframe_to_mongo(df, collection):
    
    records = df.to_dict('records')
    db[collection].insert_many(records)

def mongo_to_dataframe(collection):

    cursor = db[collection].find() 
    df = pd.DataFrame(list(cursor))
    del df['_id']
    
    return df

def clear_database():
    db.drop_collection("tipos_infracoes_2018")
    db.drop_collection("acidentes_fatais_2008_2017")
    db.drop_collecttion("pedestres_mortos_trechos_ns_2008_2017")
    db.drop_collection("cursos_oferecidos_detran_2018")
    db.drop_collection("campanhas_educativas_detran_2012_2019")

### Refresh Database

In [69]:
# Deletes all records, only if necessary
clear_database()

### Salvar e recuperar DataFrames

In [39]:
#dataframe_to_mongo(df_tipos_infracoes_2018, "tipos_infracoes_2018")
#dataframe_to_mongo(df_acidentes_fatais_2008_2017, "acidentes_fatais_2008_2017")
#dataframe_to_mongo(df_pedestres_mortos_trechos_ns_2008_2017, "pedestres_mortos_trechos_ns_2008_2017")
#dataframe_to_mongo(df_cursos_oferecidos_detran_2018, "cursos_oferecidos_detran_2018")
dataframe_to_mongo(df_campanhas_educativas_2012_2019, "campanhas_educativas_detran_2012_2019")

In [40]:
test = mongo_to_dataframe("campanhas_educativas_detran_2012_2019")
test.head()

,Campanha,Objetivos da Campanha,Data de Início da campanha,Data Final da Campanha,Local do Evento da Campanha,Região Administrativa do Evento da Campanha,Qtd de Motoristas Abordados no Evento,Qtd de Pedestres Abordados no Evento,Qtd De Motociclistas Abordados no Evento,Qtd de Ciclistas Abordados no Evento,Data Inicial do Evento da Campanha,Hora Inicial do Evento da Campanha,Data Final do Evento da Campanha,Hora Final do Evento da Campanha,Relatório do Evento,Rodovia
0,GECED - BLITZEN NOTURNAS,Campanha Educativa de entrega de folder Alcool...,2015-11-27 00:00:00.000,2015-10-27 00:00:00.000,BR 070,--,700,0,0,0,2012-01-06,00:00:00:00,2012-01-06,00:00:00:00,NaN,NaN
1,Operação Funil,"Álcool e direção, conscientização de motoristas.",2012-01-10 00:00:00.000,2012-01-10 00:00:00.000,BR 040 - Posto da Polícia Rodoviária Federal,--,300,0,0,0,2012-01-10,00:00:00:00,2012-01-10,00:00:00:00,NaN,NaN
2,Operação Funil,Abordagem de carros. Operação iniciada ás 15:3...,2012-01-12 00:00:00.000,2012-01-12 00:00:00.000,BR 251/ DF 001 - Saída para Unaí,--,448,0,0,0,2012-01-12,00:00:00:00,2012-01-12,00:00:00:00,NaN,NaN
3,Operação Funil,Abordagem de carros iniciando ás 15h e finaliz...,2012-01-19 00:00:00.000,2012-12-30 00:00:00.000,CPRV BR 060 - Samambaia,--,550,0,0,0,2012-01-19,00:00:00:00,2012-01-19,00:00:00:00,NaN,NaN
4,Operação Funil,Abordagem de carros.,2012-01-06 00:00:00.000,2012-12-30 00:00:00.000,BR 020 Posto da BPRv,--,0,0,0,0,2012-01-20,00:00:00:00,2012-01-20,00:00:00:00,NaN,NaN
